## Negative Conversion

우리는 이 작업을 python을 이용해 하고 있기 때문에, positive conversion의 경우, python의 `ast` 모듈을 이용해서 비교적 쉽게 할 수 있었다.

하지만, converted된 typescript annotation의 경우, `ast` 모듈을 이용한 분석은 불가능해진다.

이를 여전히 python 내에서 python annotation으로 convert 해보도록 한다.

In [1]:
from typing import Dict
from crimson.py_ts_types_beta.convert_typing.negative import tokenize_and_extract_strings, convert_unit_node, convert_first_unitnode

`ast.parse`를 쓰면, 저 annotation 안에 포함된 string들은 알아서 string으로 잘 분류되고, 그 안의 `[` 나 `<` 같은 요소들은 noise로 간주하지 않아도 된다.

하지만 negative conversion에서는 parser의 도움을 받지 못 한다. 그래서 먼저 저 string들을 token처리 해야한다.

``` ts
Union<int|List<int>|Union<Tuple<[Dict<str,Dict<str,int>>,Literal<'[bye>'|2|'[hi]'>]>|int>>
```

In [2]:
ts_annotation = "Union<int|List<int>|Union<Tuple<[Dict<str,Dict<str,int>>,Literal<'[bye>'|2|'[hi]'>]>|int>>"

In [3]:
processed, extracted_strings, used_quotes = tokenize_and_extract_strings(ts_annotation)
processed, extracted_strings, used_quotes 

('Union<int|List<int>|Union<Tuple<[Dict<str,Dict<str,int>>,Literal<text0|2|text1>]>|int>>',
 ['[bye>', '[hi]'],
 ["'", "'"])

`processed`:

``` ts
Union<int|List<int>|Union<Tuple<[Dict<str,Dict<str,int>>,Literal<text0|2|text1>]>|int>>
```

`tokenize_and_extract_strings` 사용 후, `processed`에 더 이상 string이 없는 것을 볼 수 있다. 

`processed`와 함께 생성된 `extracted_strings`, `used_quotes` 들은 이후 tokenized된 string들을 회복시킬 때 사용된다.

`convert_unit_node` 함수는 내부에 더 이상 convert 할 element가 없는 annotation(`unit_node`)을 안전하게 convert 한다. 예를들어,

`Dict<str,int>` 를 잘 convert 할 수 있다.
`Dict<str,Dict<str,int>>` 는 아마 문제가 생길 수 있다.


In [4]:
convert_unit_node('Dict', 'Dict<str,int>')

'Dict[str,int]'

`convert_first_unitnode` 는 현재 annotation에서 처음 나오는 `unit_node`를 convert 한다.

In [5]:
convert_first_unitnode('Dict<str,Dict<str,int>>')

'Dict<str,Dict[str,int]>'

negative conversion에서는 `<` 와 `>`가 node 구성의 기본이 됨으로, conversion된 `Dict[str,int]` 부분은 더 이상 node로 취급되지 않는다.
그래서 위와같이 한차례 convert된 string을 또 다시 convert하면, 이제는 전체 annotation이 하나의 unit_node임으로, 최종적인 converted annotation을 얻게 된다.

In [6]:
convert_first_unitnode('Dict<str,Dict[str,int]>')

'Dict[str,Dict[str,int]]'

In [7]:
Dict[str,Dict[str,int]]

typing.Dict[str, typing.Dict[str, int]]

즉, negative conversion은 저 `convert_first_unitnode`를 더 이상 convert할 필요가 없을 때까지 계속 사용함으로써 완료된다.

물론 그렇게 conversion이 완료된 annotation의 tokenized된 string을 복원해줌으로써, 전체적인 과정이 비로서 완료된다.